<a href="https://colab.research.google.com/github/LeoCboi/stock_market_analysis_by_python/blob/main/Stock_Report_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경설정

In [ ]:
 !pip install pykrx

In [ ]:
# (라이브러리 설치 후 런타임 재시작 필요)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Matplotlib 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic')   

-----

## 주가 가져오기(삼성전자)

In [ ]:
from pykrx import stock

start_date = "20200101"
end_date = "20230420"
today = "20230420"
ticker = "005935" # LG에너지 솔루션
df = stock.get_market_ohlcv(start_date, end_date, ticker)
df.head(3)

## 이동평균 구하기

In [ ]:
df['ma5'] = df['종가'].rolling(5).mean()
df['ma20'] = df['종가'].rolling(20).mean()
df['ma100'] = df['종가'].rolling(100).mean()
df['ma200'] = df['종가'].rolling(200).mean()

In [ ]:
df.head()

## 이동평균 시각화

In [ ]:
import plotly.express as px
df1 = df[['종가', 'ma5', 'ma20', 'ma100', 'ma200']]
fig = px.line(df1,  y=df1.columns,
              title='이동평균선')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")
fig.show()

이동평균 데이터를 분석하였을 때 코로나 초기에 주식 가격이 폭락하였으나, 21년 초에 회복하여 현재는 코로나이전의 곡선으로 돌아왔음.

# [스토캐스틱(Stochastic)]( https://ko.wikipedia.org/wiki/%EC%8A%A4%ED%86%A0%EC%BA%90%EC%8A%A4%ED%8B%B1)
- 최근 N일간의 최고가와 최저가의 범위 내에서 현재 가격의 위치를 표시
- **매수세** > 매도세: 현재 가격의 위치가 높게 형성됨
- **매도세** > 매수세: 현재 가격의 위치가 낮게 형성됨

$$스토캐스틱N = \frac{(현재 가격 - N일 중 최저가)}{(N일 중 최고가 - N일중 최저가)}$$

- 스토캐스틱 값의 범위는 항상 0~100% 사이
    - 스토캐스틱 = **100%**: 현재 가격이 N일간 최고가, 매수세가 강함
    - 스토캐스틱 = **0%**: 현재 가격이 N일간 최저가, 매도세가 강함함
    - 스토캐스틱 >= **80%**: 과매수 상태(하락 가능성이 커짐),
    - 스토캐스틱 <= **20%**: 과매도 상태(상승 가능성이 커짐)
    

----
(예)
```
최근 5일간 최고가가 15,000원이고 최저가가 10,000원인 주식
```
- 현재가가 14,000원이라면?
    - 매수세가 강하여 오르는 추세임
    - 스토캐스틱 값은 80%
- 현재가가 11,000원이라면?
    - 매도세가 강하여 내리는 추세임
    - 스토캐스틱 값은 20%
----

## Fast Stochastic 공식


$$Fast \%K = \frac{(현재 가격 - N일 중 최저가)}{(N일 중 최고가 - N일중 최저가)}$$
<br>
$$Fast \%D = Fask \%K의 m기간 이동평균$$
<br>
- 일반적으로 N=5. m=3을 사용
- N을 10으로 사용할 경우, m=6으로 사용
 



## Slow Stochastic 공식

$$Slow\%K = Fast\%K의 m기간 이동평균$$
<br>
$$Slow\%D = Slow\%K의 t기간 이동평균$$
<br>

- Slow Stochastic에서는 n(5)-m(3)-t(3) 공식이 가장 많이 사용
- 네이버금융은 n(15)-m(5)-t(3)을 사용

- Fast Stochastic은 그래프의 변화가 너무 잦고 급격하여 노이즈 즉 가짜 신호가 많아 매수 매도시 참고하기 어렵다는 문제점이 있기 때문에 일반적으로 Slow Stochastic을 사용

 


 

## 스토캐스틱 구하기

In [ ]:
# Fast %K = ((현재가 - n기간 중 최저가) / (n기간 중 최고가 - n기간 중 최저가)) * 100
def get_stochastic_fast_k(close_price, low, high, n=5):
  fast_k = ((close_price - low.rolling(n).min()) / (high.rolling(n).max() - low.rolling(n).min())) * 100
  return fast_k
 
# Slow %K = Fast %K의 m기간 이동평균
def get_stochastic_slow_k(fast_k, n=3):
  slow_k = fast_k.rolling(n).mean()
  return slow_k
 
# Slow %D = Slow %K의 t기간 이동평균
def get_stochastic_slow_d(slow_k, n=3):
  slow_d = slow_k.rolling(n).mean()
  return slow_d 

스토캐스틱 구하기

In [ ]:
# fast_k, slow_k, slow_d를 획득
df['fast_k'] = get_stochastic_fast_k(df['종가'], df['저가'], df['고가'], 5)
df['slow_k'] = get_stochastic_slow_k(df['fast_k'], 3)
df['slow_d'] = get_stochastic_slow_d(df['slow_k'], 3)

## 스토캐스틱 시각화

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Line(x = df.index, y = df['종가'], name='종가'),
    secondary_y= True
)

# Add traces
fig.add_trace(
    go.Line(x = df.index, y = df['slow_k'], name='slow_k'),
    secondary_y= False
)

# Add traces
fig.add_trace(
    go.Line(x = df.index, y = df['slow_d'], name='slow_d'),
    secondary_y= False
)

# Add traces
fig.add_trace(
    go.Line(x = df.index, y=[80] * len(df.index), name='80선'),
    secondary_y= False
)

# Add traces
fig.add_trace(
    go.Line(x = df.index, y=[20] * len(df.index), name='20선'),
    secondary_y= False
)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

# Set y-axes titles
fig.update_yaxes(title_text="<b>stocastic</b> axis", secondary_y=False)
fig.update_yaxes(title_text="<b>종가</b> axis", secondary_y=True)

fig.show()

## 골든크로스
- 단기이동평균선이 장기이동평균선을 뚫고 상승하는 것
- 스토캐스틱 20 이하에서 %K선이 %D선을 상향 돌파
- 매수 시점

In [ ]:
df.query('(slow_k >= slow_d) and (slow_k <= 20)')

## 데드크로스
- 단기이동평균선이 장기이동평균선을 뚫고 하락하는 것
- 스토캐스틱 80 이상상에서 %K선이 %D선을 하향 돌파
- 매도 시점


In [ ]:
df.query('(slow_k <= slow_d) and (slow_k >= 80)')

# 그래프를 통한 주가 분석

## 환경설정 / 야후파이낸스를 불러오지 않음.

In [ ]:
!pip install pandas-datareader
!pip install yfinance

In [ ]:
from pandas_datareader import data
import yfinance as yfin

yfin.pdr_override()

df = data.get_data_yahoo('005935.KS', start = '2020-01-01')
df.tail()

In [ ]:
df['Close'].plot(figsize = (18, 5)) #거래일 종가에 대한 거래금액 시각화화

In [ ]:
import plotly.express as px

fig = px.line(df, y = 'Close')
fig.show()

In [ ]:
fig = px.bar(df, y='Volume')
fig.show()

In [ ]:
fig = px.line(df, y='Close', title='Time Series with Rangeslider')

fig.update_xaxes(rangeslider_visible = True)
fig.show()

In [ ]:
fig = px.line(df, y='Close', title='Time Series with Rangeslider')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector = dict(
        buttons = list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Line(x = df.index, y = df['Close']),
    secondary_y= False
)

# Add traces
fig.add_trace(
    go.Bar(x = df.index, y = df['Volume']),
    secondary_y= True
)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

# Set y-axes titles
fig.update_yaxes(title_text="Close axis", secondary_y=False)
fig.update_yaxes(title_text="Volume axis", secondary_y=True)

fig.show()

#Candlestick을 통한 삼성전자(우) 데이터 분석

In [ ]:
import pandas as pd
from datetime import datetime

fig = go.Figure(data = go.Candlestick(
        x = df.index,
        open = df['Open'],
        close = df['Close'],
        low = df['Low'],
        high = df['High'],
))

fig.show()

총 일자별 등락률과 거래양을 알 수 있는데, 채권 이자율이 낮을 경우나 이자율이 예상이 갈 경우에는 시장의 반응은 상승으로 가져갔다. 이와 반대일 경우에는 박스권을 형성하면서 서서히 낮아졌다. 또한, 반도체 수급이 낮을 경우에는 단기적인 상승렐리를 이어왔으나, 잇다른 국제문제와 경기침체로 인한 반도체 재고가 쌓이면서 영업이익이 계속해서 줄어들었다. 또한, 신제품에 대한 기대치가 낮아지면서 주가는 코로나 이전의 상태로 돌아갔다.

#경쟁사 데이터 분석(반도체 및 첨단디바이스 제작 관련 업체로 한정)

In [ ]:
bank_tickers = ['005935.KS',
                'AAPL', # APPLE
                'TSMC34.SA', # TSMC
                '000660.KS', # SK 하이닉스
                'NVDA' #엔비디아
                 ]
df = pd.DataFrame()
for ticker in bank_tickers:
    temp = data.get_data_yahoo(ticker, start = '2020-01-01')['Close']
    df = pd.merge(df, temp, right_index=True, left_index=True, how='outer', suffixes=('', ticker))

# 중국꺼는 못불어오는 거 같음.

In [ ]:
df

In [ ]:
df.set_axis(labels=bank_tickers, axis=1, inplace = True) # Date를 제외하고 티커로 컬럼명 변경
df.head()

## 5개 경쟁사 시각화

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df.values)
scaled_df = pd.DataFrame(data=scaled_data, columns=df.columns, index = df.index)

In [ ]:
fig = px.line(scaled_df, x=scaled_df.index, y=scaled_df.columns)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

In [ ]:
import plotly.express as px
fig = px.line(df, x=df.index, y=df.columns)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()
     

# 삼성전자(우)에 대한 상승기에서의 불타기 / 하락기에서의 물타기 적용 해보기

## 상승기 때의 불타기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pandas-datareader
!pip install yfinance

In [ ]:
from pandas_datareader import data
import yfinance as yfin
import pandas as pd
from datetime import timedelta
import plotly.graph_objects as go

yfin.pdr_override()

def get_stocks(ticker, start = '2020-01-01', end = '2022-01-01'):
    stock_df = data.get_data_yahoo(ticker, start = start)
    #stock_df['Close'].plot()
    return stock_df

def get_price_dataframe(num, percent, start_price):
    price_list = [start_price]
    average_price_list = []
    for i in range(num):  #몇 번 반복할 것인지를 결정하는 부분
        price_list.append(price_list[i] * (1 + percent)) # 00%가 오를 때 몇번을 살 것인지를 결정하는 로직
    df = pd.DataFrame({'매수가': price_list})
    df['매수량'] = range(len(price_list))#1 #한 주씩 사는 것으로 셋팅하였으며, range(len(price_list)로 로직을 구성할 경우, 기간에 따라 1주씩 늘어남.
    df['보유량'] = df['매수량'].cumsum() #누적값을 확인하는 방법
    average_price_list = [df['매수가'][0]]
    for i in range(len(df)-1):
        average_price = (average_price_list[i] * df['보유량'][i] + df['매수가'][i+1] * df['매수량'][i+1]) / df['보유량'][i+1]
        average_price_list.append(average_price)
    df['매수 후 평균단가'] = average_price_list
    df['원금'] = df['매수 후 평균단가'] * df['보유량'] 
    df['매수 후 평가금액'] = df['매수가'] * df['보유량'] 
    df['수익률'] = df['매수 후 평가금액']/df['원금'] - 1
    return df

def get_sell_dates(plan_df, stock_df):
    first_date = []
    for value in plan_df['매수가']:
        # print(value)
        index_list = stock_df.query(f'Close >={value} and Close <={value+1}' ).index

        if len(index_list) == 0 or (len(first_date) > 0 and index_list[-1] - first_date[-1] >= timedelta(days=5)
    ):
            index_list = stock_df.query(f'Close <={value+1}' ).index

        # print(index_list)

        if len(first_date) == 0:
            first_date.append(index_list[0])
        else:
            for i in index_list:
                if i > first_date[-1]:
                    # print('i: {}, first_date:{}'.format(i, first_date[-1]))
                    first_date.append(i)
                    break
        # print(first_date[-1])
        # print('-'*10)
    return first_date

def plot_status(stock_df, plan_df, sell_dates):

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=stock_df.index,
        y=stock_df['Close'],
        name = '종가',
        # connectgaps=True # override default to connect the gaps
    ))
    fig.add_trace(go.Scatter(
        x=stock_df.index,
        y=[plan_df['매수 후 평균단가'].iloc[-1]] * len(stock_df.index),
        name='매수 후 평균단가',
    ))
    fig.add_trace(go.Scatter(
        x=sell_dates,
        y=plan_df['매수가'],#TSLA_df.loc[first_date]['Close'],
        mode="markers+text",
        name="분할매수시점",
        # text=plan_df['매수가'],#["Text D", "Text E", "Text F", '3', '3'],
        textposition="bottom center"
    ))
    fig.show()

def run():#ticker, num, percent, start_price):
    ticker = input('시뮬레이션 할 종목의 티커를 입력하세요.(예:TSLA): ')
    stock_df = get_stocks(ticker)
    num = input('몇 번 분할매수를 진행하시겠습니까? (예:20): ')
    percent = input('몇 % 상승시 분할매수를 진행하시겠습니까? (예:1): ')
    print('{}의 종가 최고가는 {:.2f}입니다.'.format(ticker, stock_df['Close'].max()))
    start_price = input('첫 주가 구매 금액을 얼마로 설정하시겠습니까?(예:214): ')
    
    plan_df = get_price_dataframe(int(num), int(percent)/100, float(start_price))
    sell_dates = get_sell_dates(plan_df, stock_df)

    print('\n\n')
    print('='*90)
    print('{}의 현재 보유량은 {}, 매수 후 평균단가는 ${:.2f}입니다.'\
      .format(ticker, plan_df.iloc[-1]['보유량'], plan_df.iloc[-1]['매수 후 평균단가']))

    result = (stock_df.iloc[-1]['Close'] * plan_df.iloc[-1]['보유량'] / plan_df.iloc[-1]['원금'] -1) * 100
    print('총 투자 원금은 ${:.2f} 입니다. {} 기준 수익률은 {:.2f}%입니다.'\
        .format(plan_df.iloc[-1]['원금'], stock_df.index[-1], result))    
    
    filename = '{}_planner.xlsx'.format(ticker)
    plan_df.to_excel(filename)
    print('='*90)
    print('※{}이 생성되었습니다.좌측의 생성된 파일을 다운로드 받아주세요'.format(filename))
    plot_status(stock_df, plan_df, sell_dates)

In [ ]:
run()

너도나도 사기 좋을 때인 20년 초에서부터 4.3만에 주식거래를 하고, 21년 1월에 모두 청산했을 경우 

##하락기일 때 물타기

In [ ]:
!pip install pandas-datareader
!pip install yfinance
!curl -L -O https://raw.githubusercontent.com/zzhining/real_data/main/mool_simulator.py

In [ ]:
import mool_simulator as sim
sim.run()

# 삼성전자(우) 관련 신문기사 스크랩

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime

title_list = []
url_list = []
article_list = []
date_list = []

for i in range(1):
    date= datetime.date.today() - datetime.timedelta(days = i)
    url = f'https://finance.naver.com/news/mainnews.naver?date={date}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')

    titles = soup.find_all('dd', {'class': 'articleSubject'})
    for t in titles:  
        title = t.text.strip()
        url = t.find('a').get('href')
        response = requests.get('https://finance.naver.com/' + url) # 상세 기사 페이지
        soup = BeautifulSoup(response.text, 'lxml')
        article = soup.find('div', {'class': 'articleCont'})
        
        title_list.append(title)
        url_list.append(url)
        if article is not None:
            article_list.append(article.text.strip())
        else:
            article_list.append('')
        date_list.append(date)

df = pd.DataFrame({'기사제목': title_list, '본문url': url_list, '기사본문': article_list, '날짜': date_list})
df.to_csv('article.csv', encoding = 'euc-kr')

In [ ]:
df.head()

#워드클라우드 생성


In [ ]:
# 한국어 텍스트 전처리를 위한 konlpy 패키지 설치
!pip install konlpy

In [ ]:
# 한글 폰트(나눔 폰트) 설치
!sudo apt-get install -y fonts-nanum

In [ ]:
# 워드 클라우드를 만들기 위한 작업
import matplotlib.pyplot as plt
from collections import Counter
from konlpy.tag import Okt

article = df['기사본문'][0]
article

In [ ]:
# 기사의 내용을 토큰으로 만들기
okt = Okt()
tokens = okt.nouns(article)
counted_tokens = Counter(tokens)
top_20 = counted_tokens.most_common(20)
print(top_20)

In [ ]:
from wordcloud import WordCloud
wc = WordCloud(background_color='white', font_path='NanumBarunGothic.ttf')
wc.generate_from_frequencies(counted_tokens)
figure = plt.figure()
figure.set_size_inches(10, 10)
ax = figure.add_subplot(1, 1, 1)
ax.axis("off")
ax.imshow(wc)